In [116]:
import pandas as pd
import numpy as np

In [117]:
link_df = pd.read_csv("links.csv", index_col=0)
link_df.head()

,from,to,value,from_id,to_id
0,慶應義塾大学,研究,0.025952,4,1405
1,慶應義塾大学,学部,0.022282,4,439
2,慶應義塾大学,月,0.020959,4,2603
3,慶應義塾大学,学生,0.020782,4,1603
4,慶應義塾大学,情報,0.019950,4,2291


In [118]:
node_df = pd.read_csv("nodes.csv", index_col=0)
node_df.head(6)

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me,index
0,東京大学,0,2,2,2,2,0,True,False,False,False,False,0
1,京都大学,2,0,2,2,2,0,False,True,False,False,False,1
2,明治大学,2,2,2,2,0,0,False,False,False,False,True,2
3,早稲田大学,2,2,0,2,2,0,False,False,True,False,False,3
4,慶應義塾大学,2,2,2,0,2,0,False,False,False,True,False,4
5,PDF,2,2,3,3,2,2,True,True,False,False,True,5


In [119]:
node_df[node_df.shortest_step == 0]

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me,index
0,東京大学,0,2,2,2,2,0,True,False,False,False,False,0
1,京都大学,2,0,2,2,2,0,False,True,False,False,False,1
2,明治大学,2,2,2,2,0,0,False,False,False,False,True,2
3,早稲田大学,2,2,0,2,2,0,False,False,True,False,False,3
4,慶應義塾大学,2,2,2,0,2,0,False,False,False,True,False,4


In [120]:
node_ids = node_df['index'].values

In [164]:
from_ids =  np.array(link_df.from_id)
to_ids = np.array(link_df.to_id)
distances = np.array(node_df.shortest_step)
from numpy.random import *
start_positions = uniform(-2,2, (len(node_ids), 3))
start_positions[0] = np.array([-1,1,1])
start_positions[1] = np.array([-1,-1,-1])
start_positions[2] = np.array([1,1,-1])
start_positions[3] = np.array([1,-1,1])
start_positions[4] = np.array([-1,1,-1])
import copy
positions = copy.deepcopy(start_positions)

In [178]:
import copy
positions = copy.deepcopy(start_positions)
for i in range(500): 
    Fs = [0]* len(node_ids)
    for node_id in node_ids:
        if node_id not in [0, 1, 2, 3, 4]:
            P = positions[node_id]
            F = np.array([0, 0, 0])
            linked_node_ids = to_ids[np.where(from_ids == node_id)]
            for node_i_id in linked_node_ids:
                D_i = float(distances[node_i_id] + 1)
                P_i = positions[node_i_id]
                x_i = (P_i - P) - ((P_i - P) / np.linalg.norm(P_i - P)) # バネの伸びベクトル(バネのもとの長さは1)
                F_i = x_i * (1/D_i)  # Nに働く力ベクトルを算出(バネ定数はD_iの逆数)
                F = F + F_i
            Fs[node_id] = F * D_i * 0.001
    for node_id in node_ids:
        positions[node_id] = positions[node_id] + Fs[node_id]
    if i % 10 == 0:
        print(np.sum(np.absolute(np.array(Fs[5:]))))

45.838887347953
31.773256426432834
24.181792167247373
19.251414823820753
15.824890975963985
13.365993048041856
11.567773360713296
10.218251172411307
9.169922603464045
8.342215750849075
7.6796917058561815
7.139656394356598
6.690936731121502
6.313383177549222
5.988692668772737
5.706552203738166
5.45669136606004
5.2346010575892885
5.034833749609819
4.853761655526552
4.688250017401932
4.53607727852121
4.3957327251923015
4.265175284518513
4.1430598466279935
4.028565985223818
3.9206167386712285
3.818407471286812
3.721810391421567
3.6301631014619415
3.5432522668389246
3.460850456184152
3.382919859630023
3.30864656722366
3.2383450289257856
3.1711710861788
3.1067878845855827
3.045158203571824
2.9861774138584645
2.9295956782163484
2.8750320726868774
2.82242998937855
2.771601648390567
2.722461518267226
2.6750753967459144
2.6292419059160816
2.5848119426610636
2.5416959305460147
2.499963365756646
2.4594846276319347


In [181]:
# pd.DataFrame(positions, columns=['x','y','z']).to_csv('normal.csv')
pd.DataFrame(positions, columns=['x','y','z']).to_csv('takeuchi.csv')

In [179]:
start_positions

array([[-1.        ,  1.        ,  1.        ],
       [-1.        , -1.        , -1.        ],
       [ 1.        ,  1.        , -1.        ],
       ...,
       [ 0.6928154 ,  1.68818213, -0.9419346 ],
       [ 0.66579461, -0.43664928, -1.49406485],
       [ 1.44154426,  1.92945304, -0.58298092]])

In [180]:
positions

array([[-1.        ,  1.        ,  1.        ],
       [-1.        , -1.        , -1.        ],
       [ 1.        ,  1.        , -1.        ],
       ...,
       [ 0.36739009,  1.24925322, -0.72338481],
       [ 0.45182163, -0.13944754, -0.7450748 ],
       [ 0.05837693,  0.54831132, -0.02332937]])